Early simple rule-based model

imports

In [ ]:
def separate(sentence):
  #takes in a sentence, splits it into the subsentences
  tokens = sentence.split() # or  nltk.word_tokenize(sentence)
  current = []
  sentences = []
  for i, word in enumerate(tokens, 0):
    if word in ["then", "followed"]:
      sentences.append(' '.join(current))
      current = []
    elif word == "and" and tokens[i+1] in ['a', 'an', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']:
      sentences.append(' '.join(current))
      current = []
    else:
      current.append(word)
    sentences.append(' '.join(current))
    return sentences


Neural network based model

This is based on the code we used in the sentence to waypoint models

Imports

In [ ]:
!pip install transformers
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelWithLMHead
from matplotlib import pyplot as plt
#from transformers import BertModel, BertConfig
from google.colab import files
files.upload()  # load dataGeneration.py

from splittingDataGenerator import generateTestBatch

In [ ]:
# import the pretrained models
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelWithLMHead.from_pretrained("bert-base-uncased")
baseModel = model.bert
#config = BertConfig()
#baseModel = BertModel(config)

# helper for padding 2D lists into tensors
def listToTensor(list):
  maxLen = max(len(l) for l in list)
  tensor = torch.tensor([l + (maxLen - len(l)) * [0] for l in list])
  return tensor


In [ ]:
# custom head for BERT
# credit to Martin Zablocki: https://zablo.net/blog/post/custom-classifier-on-bert-model-guide-polemo2-sentiment-analysis/
class SplittingNet(nn.Module):
  def __init__(self, base_model, n_classes=30, base_model_output_size=768, hidden_size = 100, dropout=0.05):
    super().__init__()
    #self.base_model = base_model
    self.bert = base_model
    #self.linear = nn.Linear(hidden_size, 30)
    #self.dropout = nn.Dropout(dropout)
    self.lstm = nn.LSTM(input_size=base_model_output_size, hidden_size=hidden_size,
                        num_layers=2, dropout=dropout)
    #self.lstm2 = nn.LSTM(input_size)
    self.linear = nn.Linear(100, 30)
    self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(base_model_output_size, hidden_size),
            nn.PReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, n_classes),
            nn.PReLU()
        )
    
    for layer in self.classifier:
            if isinstance(layer, nn.Linear):
                layer.weight.data.normal_(mean=0.0, std=0.02)
                if layer.bias is not None:
                    layer.bias.data.zero_()

  def forward(self, input, *args):
    X, attention_mask = input
    hidden_states = self.bert(X, token_type_ids=None, attention_mask = attention_mask)
    #print(hidden_states.last_hidden_state.shape)
    
    #output = self.classifier(hidden_states.last_hidden_state[:,0,:])
    output = self.lstm(hidden_states.last_hidden_state.permute(1,0,2))[0].permute(1,0,2)
    #print(output.shape)
    output = self.linear(output[:,-1,:])
    #print(output.shape)
    return output
    

  #def forward(self, input, *args):
  #  X, attention_mask = input
  #  hidden_states = self.bert(X, attention_mask = attention_mask)[0]
  #  lstm_out = self.lstm(hidden_states.permute(1,0,2))[0].permute(1,0,2)
  #  return self.linear(lstm_out[:, -1, :])

This is where it trains

In [ ]:
# training model
if torch.cuda.is_available():  
  #device = torch.device("cpu")
  device = torch.device("cuda:0") 
else:  
  device = torch.device("cpu")
print(device)

net = SplittingNet(baseModel)
net = net.to(device=device)
criterion = nn.MultiLabelSoftMarginLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
objectiveValues = []
running_loss = 0

# each epoch is a newly generated batch of test data
for epoch in range(7000):
  text, labels = generateTestBatch(100)
  labels = torch.tensor(labels, dtype = int)

  enc = tokenizer.batch_encode_plus(text)
  X = listToTensor(enc["input_ids"])
  attn = listToTensor(enc["attention_mask"])

  X = X.to(device)
  attn = attn.to(device)
  labels = labels.to(device)
  #labels=labels.to(torch.float32)
  #labels.type(torch.LongTensor)
  # zero the parameter gradients
  optimizer.zero_grad()

  # forward + backward + optimize
  outputs = net((X, attn))
  if epoch == 0:
    print(outputs.shape)
    print(labels.shape)
  #print(outputs.shape)
  #outputs.type(torch.LongTensor)
  #print(labels.shape)
  loss = criterion(outputs, labels)
  #loss.type(torch.LongTensor)
  loss=loss.to(torch.float32)
  loss.backward()
  optimizer.step()

  # print statistics
  running_loss += loss.item()
  if ((epoch + 1) % 100 == 0):
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / 100))
    running_loss = 0
  objectiveValues.append(loss.item())

plt.plot(objectiveValues)

In [ ]:
for epoch in range(7000):
  text, labels = generateTestBatch(100)
  labels = torch.tensor(labels, dtype = int)

  enc = tokenizer.batch_encode_plus(text)
  X = listToTensor(enc["input_ids"])
  attn = listToTensor(enc["attention_mask"])

  X = X.to(device)
  attn = attn.to(device)
  labels = labels.to(device)
  #labels=labels.to(torch.float32)
  #labels.type(torch.LongTensor)
  # zero the parameter gradients
  optimizer.zero_grad()

  # forward + backward + optimize
  outputs = net((X, attn))
  if epoch == 0:
    print(outputs.shape)
    print(labels.shape)
  #print(outputs.shape)
  #outputs.type(torch.LongTensor)
  #print(labels.shape)
  loss = criterion(outputs, labels)
  #loss.type(torch.LongTensor)
  loss=loss.to(torch.float32)
  loss.backward()
  optimizer.step()

  # print statistics
  running_loss += loss.item()
  if ((epoch + 1) % 100 == 0):
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / 100))
    running_loss = 0
  objectiveValues.append(loss.item())

plt.plot(objectiveValues)

torch.Size([101, 30])
torch.Size([101, 30])
[100] loss: 0.083
[200] loss: 0.082
[300] loss: 0.082
[400] loss: 0.082
[500] loss: 0.081
[600] loss: 0.081
[700] loss: 0.081
[800] loss: 0.080
[900] loss: 0.081
[1000] loss: 0.080
[1100] loss: 0.079
[1200] loss: 0.078
[1300] loss: 0.079
[1400] loss: 0.077
[1500] loss: 0.076
[1600] loss: 0.076
[1700] loss: 0.076
[1800] loss: 0.075
[1900] loss: 0.074
[2000] loss: 0.074
[2100] loss: 0.073
[2200] loss: 0.072
[2300] loss: 0.073
[2400] loss: 0.071
[2500] loss: 0.071
[2600] loss: 0.070
[2700] loss: 0.069
[2800] loss: 0.069
[2900] loss: 0.068
[3000] loss: 0.066
[3100] loss: 0.065
[3200] loss: 0.065
[3300] loss: 0.065
[3400] loss: 0.063
[3500] loss: 0.062
[3600] loss: 0.061
[3700] loss: 0.060
[3800] loss: 0.059
[3900] loss: 0.059
[4000] loss: 0.058
[4100] loss: 0.057
[4200] loss: 0.056
[4300] loss: 0.055
[4400] loss: 0.054
[4500] loss: 0.053
[4600] loss: 0.053
[4700] loss: 0.052
[4800] loss: 0.051
[4900] loss: 0.050


KeyboardInterrupt: ignored

In [ ]:
# save model
torch.save(net.state_dict(), "splittingBERT.pth")
files.download("splittingBERT.pth")

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = 'REPLACE_WITH_YOUR_FILE_ID'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')